In [1]:
from collections.abc import Iterable
from datasets import load_dataset, concatenate_datasets 
from train_model import preprocess_dataset, preprocess_dataset_hatexplain
from transformers import AutoTokenizer
from evaluation import Evaluation

/opt/anaconda3/envs/nlu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "vinai/bertweet-base"
labels = 'original'
hate_speech = load_dataset("hate_speech_offensive")
hatexplain = load_dataset("hatexplain")

tokenizer = AutoTokenizer.from_pretrained(model_name)

hate_speech["train"] = preprocess_dataset(hate_speech["train"], tokenizer, labels)
hatexplain["train"] = preprocess_dataset_hatexplain(hatexplain["train"], tokenizer, labels)
hatexplain["validation"] = preprocess_dataset_hatexplain(hatexplain["validation"], tokenizer, labels)
hatexplain["test"] = preprocess_dataset_hatexplain(hatexplain["test"], tokenizer, labels)
bert_dataset = concatenate_datasets([hatexplain['train'], hatexplain['validation'], hatexplain['test'], hate_speech['train']])
split = bert_dataset.train_test_split(.2, seed=3463) 

Found cached dataset hate_speech_offensive (/Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5)
100%|██████████| 1/1 [00:00<00:00, 106.53it/s]
Found cached dataset hatexplain (/Users/jiayuansong/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249)
100%|██████████| 3/3 [00:00<00:00, 108.74it/s]
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading cached processed dataset at /Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5/cache-8c91b5f7b3a54cb1.arrow
Loading cached processed dataset at /Users/jiayuansong/.cache/huggingface/datasets/hate_spee

In [8]:
threshold = 0.6 
white = 'non_aae'

### Baseline

In [9]:
eval = Evaluation('test_results_base.p', split['test'], threshold, white)

In [10]:
eval.eval()

{'EOD': {'EOD_1': -0.13753706654832265,
  'EOD_2': 0.38416894264533746,
  'EOD_comb': -0.036173611531706173},
 'SPD': {'SPD_1': -0.3625594581767705,
  'SPD_2': 0.23097201555339072,
  'SPD_comb': -0.13158744262337985},
 'DI': {'DI_non': 0.0889282361273533,
  'DI_tox_1': 1.6100395071809404,
  'DI_tox_2': 0.11588726751200858,
  'DI_tox_comb': 1.153801173655725},
 'AOD': {'AOD_1': -0.32704938090103286,
  'AOD_2': 0.26389587538212333,
  'AOD_comb': -0.25185086415302704}}

### Debiased 

In [11]:
eval = Evaluation('test_results_debias.p', split['test'], threshold, white) 

In [12]:
eval.eval()

{'EOD': {'EOD_1': -0.08496125090087525,
  'EOD_2': 0.39502861230329045,
  'EOD_comb': -0.07016988674217184},
 'SPD': {'SPD_1': -0.34986220095466136,
  'SPD_2': 0.16413977592935552,
  'SPD_comb': -0.18572242502530578},
 'DI': {'DI_non': 0.08991291980928623,
  'DI_tox_1': 1.5644189826319397,
  'DI_tox_2': 0.06773925497561174,
  'DI_tox_comb': 1.2333404567877804},
 'AOD': {'AOD_1': -0.3162665390889764,
  'AOD_2': 0.22312348480397895,
  'AOD_comb': -0.3169944911097794}}